In [1]:
# Connecting to google colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
## Mounting the Drive

import os

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CS254/CS254-Project/' # change this directory to yours
GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['data', 'img', 'Alireza_Fath_Project_1.ipynb', 'Mehrdad_Mohammad_Alireza.ipynb']


In [3]:
# importing libraries

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

In [13]:
# Load the data and store it in a pandas dataframe

# path = GOOGLE_DRIVE_PATH + '/MLProjectcsvout1.csv'
path = GOOGLE_DRIVE_PATH + '/MLProjectcsvout2.csv'
# data = pd.read_csv(path, header=None, names=['FlowRate', 'VolumeFraction', 'SolarRadiation', 'ElectricalEfficiency'])
data = pd.read_csv(path, header=None, names=['FlowRate', 'VolumeFraction', 'SolarRadiation', 'ThermalEfficiency'])
data.head(n=4)

,FlowRate,VolumeFraction,SolarRadiation,ThermalEfficiency
0,40.000000,2.000000,1000,76.866311
1,16.666667,0.666667,800,73.238863
2,22.500000,2.000000,700,71.060563
3,28.333333,2.000000,1200,78.542307


In [5]:
# Evaluate the Data
data.describe()

,FlowRate,VolumeFraction,SolarRadiation,ThermalEfficiency
count,28.000000,28.000000,28.000000,28.000000
mean,29.375000,1.357143,857.142857,74.128703
std,9.532878,0.479442,157.359158,2.546085
min,10.833333,0.666667,700.000000,70.646275
25%,22.500000,1.000000,700.000000,71.579893
50%,28.333333,1.333333,800.000000,73.899215
75%,40.000000,1.750000,1000.000000,76.544313
max,40.000000,2.000000,1200.000000,78.542307


In [6]:
# set X (training data) and y (target variable)

cols = data.shape[1]
X = data.iloc[:,0:cols-1] # iloc slicing function 
y = data.iloc[:,cols-1:cols]
y=y.values.ravel()

In [7]:
X.shape, y.shape

((28, 3), (28,))

In [8]:
#Center to the mean and component wise scale to unit variance.

standardized_X = preprocessing.scale(X)

In [9]:
# Split the data into training and testing, 80% for training and 20% for testing

X_train, X_test, y_train, y_test = train_test_split(standardized_X, y, test_size=0.20, random_state=42)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((22, 3), (6, 3), (22,), (6,))

In [11]:
# Using cross-validation to randomly find the best Hyperparameters

model=SVR(kernel="rbf", gamma="auto").fit(X_train, y_train)
y_pred_before=model.predict(X_test)

# Range of the Hyperparameters

C_range = np.logspace(0, 4, 20000)
epsilon_range = np.logspace(-8, 3, 20000)

# print (C_range)
# print (epsilon_range)

# specify parameters and distributions to sample from
param_dist = {"epsilon": epsilon_range,
              "C": C_range}

# run randomized search
n_iter_search = 5000
random_search = RandomizedSearchCV(model, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=10)

random_search.fit(X_train, y_train)

# summarize the results of the random search
print(random_search.best_score_)
print(random_search.best_estimator_.epsilon)
print(random_search.best_estimator_.C)
y_pred_after=SVR(kernel="rbf", C=random_search.best_estimator_.C, gamma="auto", epsilon=random_search.best_estimator_.epsilon).fit(X_train, y_train).predict(X_test)

# comparing the model before and after finding the best hyperparameters

score_before=r2_score(y_test, y_pred_before)
score_after=r2_score(y_test, y_pred_after)

print(score_before)
print(score_after)

0.6058160547483338
2.01959954073501e-08
369.76721000068216
0.859204534099274
0.9639661129125343


In [12]:
# Comparing the predicted output versus the test output

# Mean_Value
mean_value=np.mean(y_train)
mean_array=np.ones(6)*mean_value
mean_score=r2_score(y_test, mean_array)

d = {'Predicted Output': y_pred_after, 'Mean Output': mean_value, 'Test Output': y_test }
df = pd.DataFrame(data=d)
df.loc[len(df.index)] = [score_after, mean_score, 'R2 Score']
df.head(7)


,Predicted Output,Mean Output,Test Output
0,77.067997,74.013230,76.643122
1,76.121121,74.013230,75.589158
2,71.012563,74.013230,71.060606
3,75.810723,74.013230,75.825655
4,76.095154,74.013230,76.866311
5,71.775994,74.013230,71.327765
6,0.963966,-0.049763,R2 Score
